## Testing LSTM on VIX

In [ ]:
# LSTM Model Testing on VIX Dataset
# This script loads VIX data, trains an LSTM model, and evaluates performance

# Import necessary libraries for training
import torch.optim as optim
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from models.lstm import LSTMModel

# Load VIX data
print("Loading VIX data...")
vix_data_path = "data/real/cboe_data/VIX_History.csv"
df = pd.read_csv(vix_data_path)

# Data preprocessing
print(f"Original data shape: {df.shape}")
print(f"Date range: {df['DATE'].iloc[0]} to {df['DATE'].iloc[-1]}")

# Convert DATE column to datetime and set as index
df['DATE'] = pd.to_datetime(df['DATE'])
df.set_index('DATE', inplace=True)

# Sort by date to ensure chronological order (oldest to newest)
df.sort_index(inplace=True)

# Display basic statistics
print("\nVIX Data Statistics:")
print(df.describe())

# Check for missing values
print(f"\nMissing values per column:")
print(df.isnull().sum())

# Remove any rows with missing values
df.dropna(inplace=True)
print(f"Data shape after removing NaN: {df.shape}")

# Use CLOSE price as our target variable (VIX closing values)
# VIX represents implied volatility, so we're predicting future volatility levels
target_column = 'CLOSE'
print(f"\nUsing {target_column} as target variable")
print(f"Target range: {df[target_column].min():.2f} to {df[target_column].max():.2f}")

# Create sequences function
def create_sequences(data, seq_length):
    """
    Create input sequences and targets for time series prediction
    
    Args:
        data: 1D array of time series values
        seq_length: Length of input sequences (lookback window)
    
    Returns:
        X: Input sequences [num_sequences, seq_length, 1]
        y: Target values [num_sequences, 1]
    """
    X, y = [], []
    for i in range(len(data) - seq_length):
        # Use seq_length previous values to predict next value
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

# Parameters - keep same as original code
sequence_length = 30  # Use 30 days of VIX history to predict the next day
lookforward = 7      # Note: Currently predicting 1 day ahead (can be extended to multi-step)
test_ratio = 0.2     # 20% test split (most recent data for testing)

print(f"\nModel Parameters:")
print(f"Sequence length: {sequence_length} days")
print(f"Test ratio: {test_ratio}")

# Prepare the target data
vix_values = df[target_column].values.reshape(-1, 1)  # Reshape for scaler

# Scale the data using MinMaxScaler (normalizes to 0-1 range)
print("\nScaling data...")
lstm_scaler = MinMaxScaler()
lstm_values_scaled = lstm_scaler.fit_transform(vix_values)

print(f"Scaled data range: {lstm_values_scaled.min():.4f} to {lstm_values_scaled.max():.4f}")

# Create sequences for LSTM input
print("Creating sequences...")
lstm_X, lstm_y = create_sequences(lstm_values_scaled.flatten(), sequence_length)

print(f"Total sequences created: {len(lstm_X)}")
print(f"Input shape: {lstm_X.shape}")  # [num_sequences, seq_length]
print(f"Target shape: {lstm_y.shape}")  # [num_sequences]

# Split into train and test sets (chronological split - no shuffling for time series)
lstm_test_size = int(len(lstm_X) * test_ratio)
lstm_train_size = len(lstm_X) - lstm_test_size

# Split data chronologically (train on older data, test on recent data)
lstm_X_train, lstm_X_test = lstm_X[:lstm_train_size], lstm_X[lstm_train_size:]
lstm_y_train, lstm_y_test = lstm_y[:lstm_train_size], lstm_y[lstm_train_size:]

print(f"\nData split:")
print(f"Training samples: {len(lstm_X_train)}")
print(f"Test samples: {len(lstm_X_test)}")

# Convert to PyTorch tensors and add feature dimension
lstm_X_train = torch.FloatTensor(lstm_X_train).unsqueeze(-1)  # Add feature dimension
lstm_y_train = torch.FloatTensor(lstm_y_train).unsqueeze(-1)  # Add feature dimension
lstm_X_test = torch.FloatTensor(lstm_X_test).unsqueeze(-1)    # Add feature dimension
lstm_y_test = torch.FloatTensor(lstm_y_test).unsqueeze(-1)    # Add feature dimension

print(f"Final tensor shapes:")
print(f"X_train: {lstm_X_train.shape}")  # [num_train, seq_length, 1]
print(f"y_train: {lstm_y_train.shape}")  # [num_train, 1]
print(f"X_test: {lstm_X_test.shape}")    # [num_test, seq_length, 1]
print(f"y_test: {lstm_y_test.shape}")    # [num_test, 1]

# Create data loaders for batch processing
batch_size = 32
lstm_train_dataset = TensorDataset(lstm_X_train, lstm_y_train)
lstm_test_dataset = TensorDataset(lstm_X_test, lstm_y_test)
lstm_train_loader = DataLoader(lstm_train_dataset, batch_size=batch_size, shuffle=True)
lstm_test_loader = DataLoader(lstm_test_dataset, batch_size=batch_size, shuffle=False)

print(f"Created data loaders with batch size: {batch_size}")

# Initialize model, loss function, and optimizer
input_size = 1  # Single feature (VIX close price)
hidden_size = 64  # LSTM hidden state size
num_layers = 2    # Number of LSTM layers
output_size = 1   # Predicting single value (next day VIX)

lstm_model = LSTMModel(input_size=input_size, hidden_size=hidden_size, 
                      num_layers=num_layers, output_size=output_size)
lstm_criterion = nn.MSELoss()  # Mean Squared Error loss
lstm_optimizer = optim.Adam(lstm_model.parameters(), lr=0.001)

print(f"\nModel Architecture:")
print(f"Input size: {input_size}")
print(f"Hidden size: {hidden_size}")
print(f"Number of layers: {num_layers}")
print(f"Output size: {output_size}")
print(f"Total parameters: {sum(p.numel() for p in lstm_model.parameters())}")

# Training
print(f"\nStarting training...")
num_epochs = 50
lstm_train_losses = []
lstm_start_time = time.time()

for epoch in range(num_epochs):
    lstm_model.train()  # Set model to training mode
    lstm_running_loss = 0.0
    
    # Process each batch
    for batch_idx, (inputs, targets) in enumerate(lstm_train_loader):
        # Forward pass
        lstm_outputs = lstm_model(inputs)
        lstm_loss = lstm_criterion(lstm_outputs, targets)
        
        # Backward pass and optimization
        lstm_optimizer.zero_grad()  # Clear gradients
        lstm_loss.backward()        # Compute gradients
        lstm_optimizer.step()       # Update parameters
        
        lstm_running_loss += lstm_loss.item()
    
    # Calculate average loss for this epoch
    lstm_avg_loss = lstm_running_loss / len(lstm_train_loader)
    lstm_train_losses.append(lstm_avg_loss)
    
    # Print progress every 5 epochs
    if (epoch+1) % 5 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {lstm_avg_loss:.6f}')

lstm_training_time = time.time() - lstm_start_time
print(f"Training completed in {lstm_training_time:.2f} seconds")

# Evaluation
print(f"\nEvaluating model...")
lstm_model.eval()  # Set model to evaluation mode
lstm_predictions = []
lstm_actuals = []
lstm_inference_start = time.time()

with torch.no_grad():  # Disable gradient computation for efficiency
    for inputs, targets in lstm_test_loader:
        outputs = lstm_model(inputs)
        lstm_predictions.append(outputs.numpy())
        lstm_actuals.append(targets.numpy())
    
    # Concatenate all batch predictions
    lstm_predictions = np.concatenate(lstm_predictions)
    lstm_actuals = np.concatenate(lstm_actuals)
    
    # Inverse transform to original VIX scale
    lstm_predictions = lstm_scaler.inverse_transform(lstm_predictions)
    lstm_actuals = lstm_scaler.inverse_transform(lstm_actuals)

lstm_inference_time = time.time() - lstm_inference_start

# Calculate performance metrics
lstm_mse = mean_squared_error(lstm_actuals, lstm_predictions)
lstm_rmse = np.sqrt(lstm_mse)
lstm_mae = mean_absolute_error(lstm_actuals, lstm_predictions)

# Calculate additional financial metrics for VIX prediction
mape = np.mean(np.abs((lstm_actuals - lstm_predictions) / lstm_actuals)) * 100
direction_accuracy = np.mean((np.diff(lstm_actuals.flatten()) > 0) == 
                           (np.diff(lstm_predictions.flatten()) > 0)) * 100

print(f"\nLSTM Performance Metrics on VIX:")
print(f"Test MSE: {lstm_mse:.4f}")
print(f"Test RMSE: {lstm_rmse:.4f}")
print(f"Test MAE: {lstm_mae:.4f}")
print(f"MAPE: {mape:.2f}%")
print(f"Direction Accuracy: {direction_accuracy:.2f}%")
print(f"Inference time: {lstm_inference_time:.4f} seconds")

# Create date index for predictions
# Get the corresponding dates for test predictions
test_start_idx = sequence_length + lstm_train_size
test_end_idx = test_start_idx + len(lstm_predictions)
lstm_pred_dates = df.index[test_start_idx:test_end_idx]

print(f"Prediction period: {lstm_pred_dates[0]} to {lstm_pred_dates[-1]}")

# Plot results
plt.figure(figsize=(15, 12))

# Plot 1: Training loss over epochs
plt.subplot(4, 1, 1)
plt.plot(lstm_train_losses, 'b-', linewidth=2)
plt.title('LSTM Training Loss on VIX Data', fontsize=14, fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.grid(True, alpha=0.3)

# Plot 2: Predictions vs actuals (test period only)
plt.subplot(4, 1, 2)
plt.plot(lstm_actuals.flatten(), label='Actual VIX', color='black', linewidth=2)
plt.plot(lstm_predictions.flatten(), label='LSTM Predictions', color='red', linewidth=2)
plt.title('LSTM Predictions vs Actual VIX (Test Period)', fontsize=14, fontweight='bold')
plt.xlabel('Test Sample')
plt.ylabel('VIX Level')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 3: Predictions with dates (test period)
plt.subplot(4, 1, 3)
plt.plot(lstm_pred_dates, lstm_actuals.flatten(), label='Actual VIX', 
         color='black', linewidth=2)
plt.plot(lstm_pred_dates, lstm_predictions.flatten(), label='LSTM Predictions', 
         color='red', linewidth=2, alpha=0.8)
plt.title('LSTM VIX Predictions with Dates (Test Period)', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('VIX Level')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)

# Plot 4: Full dataset context with predictions
plt.subplot(4, 1, 4)
plt.plot(df.index, df[target_column], label='Full VIX History', 
         color='blue', alpha=0.6, linewidth=1)
plt.plot(lstm_pred_dates, lstm_predictions.flatten(), label='LSTM Predictions', 
         color='red', linewidth=3)
plt.title('LSTM Predictions in Context of Full VIX History', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('VIX Level')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

# Additional analysis: Prediction errors
plt.figure(figsize=(12, 8))

# Plot error distribution
plt.subplot(2, 2, 1)
errors = lstm_actuals.flatten() - lstm_predictions.flatten()
plt.hist(errors, bins=30, alpha=0.7, edgecolor='black')
plt.title('Prediction Error Distribution', fontsize=12, fontweight='bold')
plt.xlabel('Prediction Error (Actual - Predicted)')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

# Plot error over time
plt.subplot(2, 2, 2)
plt.plot(lstm_pred_dates, errors, color='purple', linewidth=1)
plt.title('Prediction Error Over Time', fontsize=12, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Prediction Error')
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)

# Scatter plot: Actual vs Predicted
plt.subplot(2, 2, 3)
plt.scatter(lstm_actuals.flatten(), lstm_predictions.flatten(), alpha=0.6)
plt.plot([lstm_actuals.min(), lstm_actuals.max()], 
         [lstm_actuals.min(), lstm_actuals.max()], 'r--', linewidth=2)
plt.title('Actual vs Predicted VIX', fontsize=12, fontweight='bold')
plt.xlabel('Actual VIX')
plt.ylabel('Predicted VIX')
plt.grid(True, alpha=0.3)

# Residuals plot
plt.subplot(2, 2, 4)
plt.scatter(lstm_predictions.flatten(), errors, alpha=0.6)
plt.axhline(y=0, color='r', linestyle='--', linewidth=2)
plt.title('Residuals vs Predictions', fontsize=12, fontweight='bold')
plt.xlabel('Predicted VIX')
plt.ylabel('Residuals')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Store results for later comparison with other models
lstm_results = {
    'model': 'LSTM',
    'dataset': 'VIX',
    'mse': lstm_mse,
    'rmse': lstm_rmse,
    'mae': lstm_mae,
    'mape': mape,
    'direction_accuracy': direction_accuracy,
    'training_time': lstm_training_time,
    'inference_time': lstm_inference_time,
    'predictions': lstm_predictions,
    'actuals': lstm_actuals,
    'prediction_dates': lstm_pred_dates,
    'errors': errors,
    'num_parameters': sum(p.numel() for p in lstm_model.parameters())
}

print(f"\nModel Summary:")
print(f"Dataset: VIX volatility index")
print(f"Training samples: {len(lstm_X_train)}")
print(f"Test samples: {len(lstm_X_test)}")
print(f"Sequence length: {sequence_length}")
print(f"Model parameters: {lstm_results['num_parameters']}")
print(f"Best metric - RMSE: {lstm_rmse:.4f}")

# Save results (optional)
# import pickle
# with open('lstm_vix_results.pkl', 'wb') as f:
#     pickle.dump(lstm_results, f)

print(f"\nLSTM testing on VIX dataset completed successfully!")
print(f"Results stored in 'lstm_results' dictionary for comparison with other models.")